## Deep Deterministic Policy Gradient (DDPG)

In [12]:
import copy
import gym
import torch
import random
import functools

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer
from gym.wrappers import RecordVideo, RecordEpisodeStatistics




device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()



In [14]:
def create_environment(env_name):
  env = gym.make(env_name)
  env = RecordVideo(env, video_folder='./videos1', episode_trigger=lambda x: x % 50 == 0)
  return env

#### Create the gradient policy

In [15]:
class GradientPolicy(nn.Module):

  def __init__(self, hidden_size, obs_size, out_dims, min, max):
    super().__init__()
    self.min = torch.from_numpy(min).to(device)
    self.max = torch.from_numpy(max).to(device)
    self.net = nn.Sequential(
        nn.Linear(obs_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, out_dims),
        nn.Tanh()
    )
    
  def mu(self, x):
    if isinstance(x, np.ndarray):
      x = torch.from_numpy(x).to(device)
    return self.net(x.float()) * self.max

  def forward(self, x, epsilon=0):
    mu = self.mu(x)
    mu = mu + torch.normal(0, epsilon, mu.size(), device=mu.device)
    action = torch.max(torch.min(mu, self.max), self.min)
    action = action.cpu().numpy()
    return action


#### Create the Deep Q-Network

In [16]:
class DQN(nn.Module):

  def __init__(self, hidden_size, obs_size, out_dims):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(obs_size + out_dims, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),           
        nn.Linear(hidden_size, 1),
    )

  def forward(self, state, action):
    if isinstance(state, np.ndarray):
      state = torch.from_numpy(state).to(device)
    if isinstance(action, np.ndarray):
      action = torch.from_numpy(action).to(device)
    in_vector = torch.hstack((state, action))
    return self.net(in_vector.float())


In [17]:
class ReplayBuffer:

  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)

  def __len__(self):
    return len(self.buffer)
  
  def append(self, experience):
    self.buffer.append(experience)
  
  def sample(self, batch_size):
    return random.sample(self.buffer, batch_size)

In [18]:
class RLDataset(IterableDataset):

  def __init__(self, buffer, sample_size=400):
    self.buffer = buffer
    self.sample_size = sample_size
  
  def __iter__(self):
    for experience in self.buffer.sample(self.sample_size):
      yield experience

In [19]:
def polyak_average(net, target_net, tau=0.01):
    for qp, tp in zip(net.parameters(), target_net.parameters()):
        tp.data.copy_(tau * qp.data + (1 - tau) * tp.data)

In [20]:
class DDPG(LightningModule):

  def __init__(self, env_name,capacity=100_000, 
               batch_size=256, actor_lr=1e-3, 
               critic_lr=1e-3, hidden_size=256, gamma=0.99, loss_fn=F.smooth_l1_loss, 
               optim=AdamW, eps_start=2.0, eps_end=0.2, eps_last_episode=1_000, samples_per_epoch=1_000, tau=0.005):

    super().__init__()

    self.env = create_environment(env_name)

    obs_size = self.env.observation_space.shape[0]
    action_dims = self.env.action_space.shape[0]
    max_action = self.env.action_space.high
    min_action = self.env.action_space.low

    self.q_net = DQN(hidden_size, obs_size, action_dims)
    self.policy = GradientPolicy(hidden_size, obs_size, action_dims, min_action, max_action)

    self.target_policy = copy.deepcopy(self.policy)
    self.target_q_net = copy.deepcopy(self.q_net)

    self.buffer = ReplayBuffer(capacity=capacity)

    self.save_hyperparameters()

    while len(self.buffer) < self.hparams.samples_per_epoch:
      print(f"{len(self.buffer)} samples in experience buffer. Filling...")
      self.play_episode(epsilon=self.hparams.eps_start)

  @torch.no_grad()
  def play_episode(self, policy=None, epsilon=0.):
    obs = self.env.reset()
    done = False
    r = 0
    while not done:
      if policy:
        action = policy(obs, epsilon=epsilon)
      else:
        action = self.env.action_space.sample()
      self.env.render(mode='human') 
      next_obs, reward, done, info = self.env.step(action)
      r = r+reward
      exp = (obs, action, reward, done, next_obs)
      self.buffer.append(exp)
      obs = next_obs
    return r 

  def forward(self, x):
    output = self.policy(x)
    return output

  def configure_optimizers(self):
    q_net_optimizer = self.hparams.optim(self.q_net.parameters(), lr=self.hparams.critic_lr)
    policy_optimizer = self.hparams.optim(self.policy.parameters(), lr=self.hparams.actor_lr)
    return [q_net_optimizer, policy_optimizer]

  def train_dataloader(self):
    dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch)
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=256,
    )
    return dataloader

  def training_step(self, batch, batch_idx, optimizer_idx):
    epsilon = max(
        self.hparams.eps_end,
        self.hparams.eps_start - self.current_epoch / self.hparams.eps_last_episode
    )

    
    states, actions, rewards, dones, next_states = batch
    rewards = rewards.unsqueeze(1)
    dones = dones.unsqueeze(1).bool()

    

    if optimizer_idx == 0:
      state_action_values = self.q_net(states, actions)
      next_state_values = self.target_q_net(next_states, self.target_policy.mu(next_states))
      next_state_values[dones] = 0.0
      expected_state_action_values = rewards + self.hparams.gamma * next_state_values
      q_loss = self.hparams.loss_fn(state_action_values, expected_state_action_values)
      self.log_dict({"episode/Q-Loss": q_loss})
      return q_loss
    
    elif optimizer_idx == 1:
      mu = self.policy.mu(states)
      policy_loss = - self.q_net(states, mu).mean()
      self.log_dict({"episode/Policy Loss": policy_loss})
      return policy_loss
  
  def training_epoch_end(self, outputs):
    mean_reward = self.play_episode(policy=self.policy, epsilon=self.hparams.eps_start)

    polyak_average(self.q_net, self.target_q_net, tau=self.hparams.tau)
    polyak_average(self.policy, self.target_policy, tau=self.hparams.tau)
    
    self.log("episode/mean_reward", mean_reward)

In [21]:

%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

In [22]:
algo = DDPG('BipedalWalker-v3')

trainer = Trainer(
    gpus=num_gpus, 
    max_epochs=1000,
    log_every_n_steps=10
)

trainer.fit(algo)

: 

: 